# Funções Analíticas

**Funções analíticas**, também conhecidas como **funções de janela**, são uma ferramenta poderosa em SQL para um registro anexar contextos de outros registros. Isso fará sentido com os vários exemplos que demonstraremos. Embora mostremos maneiras mais simples de realizar tarefas anteriores que realizamos com subconsultas, tabelas derivadas e expressões de tabela comuns, todas essas outras abordagens que aprendemos ainda são altamente flexíveis e necessárias. Mas, como veremos, operações analíticas comuns geralmente podem ser realizadas com essas funções de janela em vez de ferramentas de subconsulta.

Vamos configurar primeiro com o banco de dados `company_operations.db`.

In [ ]:
import sqlite3
import pandas as pd 

pd.options.display.max_rows = 999

conn = sqlite3.connect('company_operations.db')
pd.read_sql("SELECT * FROM WEATHER_MONITOR LIMIT 10", conn)

## PARTITION BY

Digamos que, em cada registro `WEATHER_MONITOR`, queríamos também mostrar a `TEMPERATURE` média para o `YEAR` e o `MONTH` desse registro. Anteriormente, usávamos uma subconsulta, uma tabela derivada ou uma expressão de tabela comum para isso.

In [ ]:
sql = """
WITH temp_avgs AS (
    SELECT strftime('%Y', REPORT_DATE) AS YEAR, 
    strftime('%m', REPORT_DATE) AS MONTH,
    AVG(TEMPERATURE) AS AVG_TEMP 
    FROM WEATHER_MONITOR
    GROUP BY 1, 2
) 

SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 
TEMPERATURE, 
AVG_TEMP

FROM WEATHER_MONITOR INNER JOIN temp_avgs

ON strftime('%Y', REPORT_DATE) = temp_avgs.YEAR
AND strftime('%m', REPORT_DATE) = temp_avgs.MONTH
"""
            
pd.read_sql(sql, conn)


Embora expressões de tabela comuns e subconsultas sejam altamente úteis e personalizáveis, essa tarefa específica é tão comum que existem funções e operadores especiais para ela. Em vez de usar todo esse trabalho de expressão de tabela comum e junção, podemos pegar a temperatura média `AVG(TEMPERATURE)` e `PARTITION` para todos os registros que compartilham o mesmo ano e mês.

In [ ]:
sql = """
SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 
TEMPERATURE, 
AVG(TEMPERATURE) OVER (PARTITION BY strftime('%Y', REPORT_DATE), strftime('%m', REPORT_DATE)) AS AVG_TEMP_Y_M

FROM WEATHER_MONITOR 

ORDER BY ID
"""
            
pd.read_sql(sql, conn, index_col='ID')


O que é particularmente poderoso em funções de janelamento como `PARTITION BY` é que podemos misturar e combinar diferentes escopos e contextos, com funções de agregação familiares como `MIN`, `MAX`, `AVG`, `SUM` e `COUNT`. Abaixo, adicionamos mais alguns campos analíticos, obtendo as temperaturas média, mínima e máxima para o `LOCATION_ID` de cada registro.

In [ ]:
sql = """
SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 
TEMPERATURE, 
AVG(TEMPERATURE) OVER (PARTITION BY strftime('%Y', REPORT_DATE), strftime('%m', REPORT_DATE)) AS AVG_TEMP_Y_M,
AVG(TEMPERATURE) OVER (PARTITION BY LOCATION_ID) AVG_TEMP_LOCATION, 
MIN(TEMPERATURE) OVER (PARTITION BY LOCATION_ID) MIN_TEMP_LOCATION,
MAX(TEMPERATURE) OVER (PARTITION BY LOCATION_ID) MAX_TEMP_LOCATION

FROM WEATHER_MONITOR 

ORDER BY ID
"""
            
pd.read_sql(sql, conn)


Também podemos reutilizar cláusulas de janela e criar alias para elas usando a palavra-chave `WINDOW`.

In [ ]:
sql = """
SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 
TEMPERATURE, 
AVG(TEMPERATURE) OVER ym AS AVG_TEMP_Y_M,
AVG(TEMPERATURE) OVER loc AVG_TEMP_LOCATION, 
MIN(TEMPERATURE) OVER loc MIN_TEMP_LOCATION,
MAX(TEMPERATURE) OVER loc MAX_TEMP_LOCATION

FROM WEATHER_MONITOR 

WINDOW ym AS (PARTITION BY strftime('%Y', REPORT_DATE), strftime('%m', REPORT_DATE)),
loc AS (PARTITION BY LOCATION_ID)

ORDER BY ID
"""
            
pd.read_sql(sql, conn)


Lembre-se de que funções de janela como `PARTITION BY` examinarão apenas registros que passarem pela condição `WHERE`. Isso significa que, se você precisar acessar registros que existem fora da condição `WHERE`, precisará voltar a usar subconsultas e expressões de tabela comuns. Observe como a inclusão de uma condição `WHERE` na consulta acima para um único `REPORT_CODE` sufocou todos os outros dados das funções de janela, tornando todos os valores estatísticos `50` em geral, já que agora há apenas um ponto de dados.

In [ ]:
sql = """
SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 
TEMPERATURE, 
AVG(TEMPERATURE) OVER ym AS AVG_TEMP_Y_M,
AVG(TEMPERATURE) OVER loc AVG_TEMP_LOCATION, 
MIN(TEMPERATURE) OVER loc MIN_TEMP_LOCATION,
MAX(TEMPERATURE) OVER loc MAX_TEMP_LOCATION

FROM WEATHER_MONITOR 
WHERE REPORT_CODE = 'UVYMMWW' 

WINDOW ym AS (PARTITION BY strftime('%Y', REPORT_DATE), strftime('%m', REPORT_DATE)),
loc AS (PARTITION BY LOCATION_ID)
"""
            
pd.read_sql(sql, conn)


## ORDER BY 

Aqui está outra aplicação útil de funções de janela. Lembre-se de que podemos usar self joins com condições de join de desigualdade para, por exemplo, obter um total contínuo de pedidos. Supondo que `CUSTOMER_ORDER_ID` reflita a data de entrada cronológica dos pedidos, posso consultar registros anteriores a cada um e somá-los como `ROLLING_QTY`.

In [ ]:
sql = """
SELECT c1.CUSTOMER_ORDER_ID, 
c1.ORDER_DATE,
c1.PRODUCT_ID,
c1.CUSTOMER_ID,
c1.QUANTITY,
SUM(c2.QUANTITY) as ROLLING_QTY

FROM CUSTOMER_ORDER c1 INNER JOIN CUSTOMER_ORDER c2
ON c1.CUSTOMER_ORDER_ID >= c2.CUSTOMER_ORDER_ID

GROUP BY 1, 2, 3, 4
"""

pd.read_sql(sql, conn)

Posso simplificar bastante isso usando uma cláusula `ORDER BY` em uma função analítica.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID, 
ORDER_DATE,
PRODUCT_ID,
CUSTOMER_ID,
QUANTITY,
SUM(QUANTITY) OVER (ORDER BY CUSTOMER_ORDER_ID) as ROLLING_QTY

FROM CUSTOMER_ORDER
"""

pd.read_sql(sql, conn)

Chega de self joins complicadas com a lógica `GROUP BY` estranha! Agora observe que, se usarmos `ORDER BY ORDER_DATE` em vez de `ORDER BY CUSTOMER_ORDER_ID`, algo estranho acontece.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID, 
ORDER_DATE,
PRODUCT_ID,
CUSTOMER_ID,
QUANTITY,
SUM(QUANTITY) OVER (ORDER BY ORDER_DATE) as ROLLING_QTY

FROM CUSTOMER_ORDER
"""

pd.read_sql(sql, conn)

Cada registro com a mesma `ORDER_DATE` tem a mesma `ROLLING_QTY`. O motivo é que `ORDER_DATE` não possui valores únicos, então cada `ORDER_DATE` soma o total de cada dia. Se quisermos totalizar arbitrariamente linha por linha, é melhor usar um campo único ordenado, como `CUSTOMER_ORDER_ID`. Mas se você ainda quiser fazer o primeiro, use a palavra-chave `ROWS BETWEEN` e especifique o intervalo.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID, 
ORDER_DATE,
PRODUCT_ID,
CUSTOMER_ID,
QUANTITY,
SUM(QUANTITY) OVER (ORDER BY ORDER_DATE ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as ROLLING_QTY
FROM CUSTOMER_ORDER
"""

pd.read_sql(sql, conn)

Tenha cuidado ao usar `ROWS BETWEEN`, pois a ordenação dos registros é arbitrária e alimenta a função, e se você reordenar os registros, obterá resultados confusos. O comportamento padrão `RANGE BETWEEN` geralmente é preferível, pois funciona com valores lógicos em vez de linhas individuais.

Também podemos criar médias móveis alterando os limites. Abaixo, criamos uma média móvel entre os 3 registros anteriores e os 3 seguintes.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID, 
ORDER_DATE,
PRODUCT_ID,
CUSTOMER_ID,
QUANTITY,
AVG(QUANTITY) OVER (ORDER BY ORDER_DATE ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING) as ROLLING_AVG
FROM CUSTOMER_ORDER
"""

pd.read_sql(sql, conn)

Vamos voltar a usar a lógica padrão `RANGE BETWEEN`. Se você quiser isolar cada registro para obter o total móvel, mas apenas dentro dos registros que compartilham `PRODUCT_ID` e `CUSTOMER_ID`, adicione `PARTITION BY` novamente. Ao analisar os registros, observe como os totais contínuos contabilizam apenas os registros que compartilham o mesmo `CUSTOMER_ID` e `PRODUCT_ID`.

In [ ]:
pd.set_option('display.max_rows', None)

sql = """
SELECT CUSTOMER_ORDER_ID, 
ORDER_DATE,
PRODUCT_ID,
CUSTOMER_ID,
QUANTITY,
SUM(QUANTITY) OVER (PARTITION BY PRODUCT_ID, CUSTOMER_ID ORDER BY ORDER_DATE) as ROLLING_QTY

FROM CUSTOMER_ORDER

ORDER BY CUSTOMER_ORDER_ID
"""

pd.read_sql(sql, conn)

## LEAD e LAG 

Duas outras funções de janela muito úteis são `LEAD()` e `LAG()`. Elas permitem recuperar o valor de outro registro com base em um campo ordenado. Abaixo, usamos `LAG()` para procurar o valor do registro anterior. Compare as colunas `QUANTITY` e `PREV_QTY` abaixo e você verá um padrão!

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID, 
CUSTOMER_ID,
ORDER_DATE, 
PRODUCT_ID,
QUANTITY,
LAG(QUANTITY, 1, 0) OVER (ORDER BY ORDER_DATE) AS PREV_QTY
FROM CUSTOMER_ORDER 
"""

pd.read_sql(sql, conn)

A função `LEAD()` procurará o próximo registro à frente.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID, 
CUSTOMER_ID,
ORDER_DATE, 
PRODUCT_ID,
QUANTITY,
LEAD(QUANTITY, 1, 0) OVER (ORDER BY ORDER_DATE) AS NEXT_QTY
FROM CUSTOMER_ORDER 
"""

pd.read_sql(sql, conn)

Você verá que o segundo e o terceiro argumentos, 1 e 0 nesses casos, controlarão o número de registros a serem procurados à frente/atrás e o valor padrão. Abaixo, alteramos a função `LAG()` para recuperar o terceiro registro atrás dele e definimos o valor padrão para `-1` se não houver nenhum para recuperar.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID, 
CUSTOMER_ID,
ORDER_DATE, 
PRODUCT_ID,
QUANTITY,
LAG(QUANTITY, 3, -1) OVER (ORDER BY ORDER_DATE) AS PREV_QTY
FROM CUSTOMER_ORDER 
"""

pd.read_sql(sql, conn)

## Classificação

A função `ROW_NUMBER()` pode ser muito útil com funções de janela para classificar itens. Por exemplo, digamos que eu queira obter os 3 produtos mais vendidos por cliente. Posso usar `ROW_NUMBER()` para atribuir um número de classificação a cada quantidade classificada por `CUSTOMER_ID` e `PRODUCT_ID`. Assim, posso filtrar apenas os três primeiros itens.

In [ ]:
sql = """
WITH TOTAL_QTYS AS (
  SELECT CUSTOMER_ID, PRODUCT_ID, SUM(QUANTITY) AS TOTAL_QTY 
  FROM CUSTOMER_ORDER 
  GROUP BY 1,2
),

PRODUCT_SALES_BY_CUSTOMER AS (
   SELECT CUSTOMER_ID, PRODUCT_ID, TOTAL_QTY,
   ROW_NUMBER() OVER (PARTITION BY CUSTOMER_ID ORDER BY TOTAL_QTY DESC) AS RANKING
   FROM TOTAL_QTYS
) 
SELECT * FROM PRODUCT_SALES_BY_CUSTOMER 
WHERE RANKING <= 3
"""

pd.read_sql(sql, conn)

`RANK()` e `DENSE_RANK()` são idênticos a `ROW_NUMBER()` em comportamento, exceto na forma como valores idênticos são tratados. Se você quiser que valores idênticos recebam a mesma classificação, use a função `RANK()` em vez de `ROW_NUMBER()`. Use `DENSE_RANK()` se quiser forçar os valores a serem consecutivos em vez de duplicatas, fazendo com que as classificações sejam ignoradas.

## Exercício

Para o período de `01/02/2024` a `28/02/2024`, insira a quantidade máxima contínua pedida (até cada `ORDER_DATE`) por `CUSTOMER_ID` e `PRODUCT_ID`. O modelo já está disponível, basta substituir o ponto de interrogação `?` abaixo.

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID,
ORDER_DATE,
CUSTOMER_ID,
PRODUCT_ID,
QUANTITY,
? as rolling_max_qty_for_customer_and_product

FROM ?
WHERE ORDER_DATE BETWEEN '2024-02-01' AND '2024-02-28'

ORDER BY CUSTOMER_ORDER_ID
"""

pd.read_sql(sql, conn)

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """
SELECT CUSTOMER_ORDER_ID,
ORDER_DATE,
CUSTOMER_ID,
PRODUCT_ID,
QUANTITY,
MAX(QUANTITY) OVER(PARTITION BY CUSTOMER_ID, PRODUCT_ID ORDER BY ORDER_DATE) as rolling_max_qty_for_customer_and_product

FROM CUSTOMER_ORDER
WHERE ORDER_DATE BETWEEN '2024-02-01' AND '2024-02-28'

ORDER BY CUSTOMER_ORDER_ID
"""

pd.read_sql(sql, conn)